In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE224705'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Defining molecular and cellular landscape behind non-response to Mycophenolate Mofetil in Systemic Lupus Erythematosus"
!Series_summary	"Lupus nephritis (LN) is one of the more severe systemic lupus erythematosus manifestations with the potential of developing into end stage kidney disease. Mycophenolate mofetil (MMF) is widely used for both induction and maintenance therapy for LN, but the one year complete renal response ranges from 30-40% in most trials. Reasons for non-response are still unknown.  Thus, anticipating lack of drug efficacy in a patient could lead to early introduction of advanced therapies.  A longitudinal cohort comprising gene-expression and clinical data of 97 MMF responder and 28 non-responder samples was retrospectively analyzed. Response to MMF was defined over time according to whether the protein/creatinine ratio in urine was reduced and maintained below 500 mg/g with the use of the treatment. Differential gene expressio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 2. Variable Availability and Data Type Conversion
# trait_row determination
trait_row = 3  # Assuming mmf_response: NO/YES can indicate Lupus trait

# age_row determination
age_row = 2  # Key 2 corresponds to 'age'

# gender_row determination
gender_row = 1  # Key 1 corresponds to 'Sex'

# Conversion functions
def convert_trait(value):
    response = value.split(': ')[1]
    return 1 if response.lower() == 'yes' else 0 if response.lower() == 'no' else None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender = value.split(': ')[1]
    return 1 if gender.lower() == 'male' else 0 if gender.lower() == 'female' else None

# Save cohort information
save_cohort_info('GSE224705', './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lupus_(Systemic_Lupus_Erythematosus)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/trait_data/GSE224705.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7029843': [0.0, 18.71, 0.0], 'GSM7029844': [0.0, 18.71, 0.0], 'GSM7029845': [0.0, 18.71, 0.0], 'GSM7029846': [0.0, 18.71, 0.0], 'GSM7029847': [0.0, 18.71, 0.0], 'GSM7029848': [0.0, 17.66, 0.0], 'GSM7029849': [0.0, 17.66, 0.0], 'GSM7029850': [1.0, 25.41, 0.0], 'GSM7029851': [1.0, 25.41, 0.0], 'GSM7029852': [1.0, 25.41, 0.0], 'GSM7029853': [1.0, 25.41, 0.0], 'GSM7029854': [1.0, 25.41, 0.0], 'GSM7029855': [1.0, 25.41, 0.0], 'GSM7029856': [1.0, 25.41, 0.0], 'GSM7029857': [0.0, 25.77, 0.0], 'GSM7029858': [0.0, 25.77, 0.0], 'GSM7029859': [0.0, 15.56, 0.0], 'GSM7029860': [0.0, 15.56, 0.0], 'GSM7029861': [0.0, 19.38, 0.0], 'GSM7029862': [0.0, 19.38, 0.0], 'GSM7029863': [0.0, 24.11, 0.0], 'GSM7029864': [0.0, 24.11, 0.0], 'GSM7029865': [0.0, 24.11, 0.0], 'GSM7029866': [1.0, 22.25, 0.0], 'GSM7029867': [1.0, 22.25, 0.0], 'GSM7029868': [1.0, 22.25, 0.0], 'GSM7029869': [0.0, 45.79, 1.0], 'GSM7029870': [0.0, 45.79, 1.0], 'GSM7029871': [0.0, 22.26, 0.0], 'GSM7029872': [0.0, 22.26, 0.0], 'GSM70298

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Determine the keys for identifiers and gene symbols
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    
    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # 3. Apply the mapping to get the gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE224705.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Lupus_(Systemic_Lupus_Erythematosus)')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE224705', './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/GSE224705.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Lupus_(Systemic_Lupus_Erythematosus)', the least common label is '0.0' with 28 occurrences. This represents 22.40% of the dataset.
The distribution of the feature 'Lupus_(Systemic_Lupus_Erythematosus)' in this dataset is fine.

Quartiles for 'Age':
  25%: 18.71
  50% (Median): 25.41
  75%: 28.97
Min: 12.39
Max: 60.05
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 14 occurrences. This represents 11.20% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json
